<a href="https://colab.research.google.com/github/SamurAIGPT/langchain-course/blob/main/chains/Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

Langchain supports a bunch of chains each with it's own functionality. The easiest of these chains is the LLMChain . It works by taking a user's input, passing in to the first element in the chain — a PromptTemplate — to format the input into a particular prompt.

### LLMChain

LLMChain is most basic chain in Langchain which we will talk about.

It takes in a prompt template, formats it with the user input and returns the response from an LLM.

#### What is the advantage of using this chain ?

Using templates as input we can dynamically change what input we provide to LLM and thus use it directly without writing from scratch. Let's discuss LLMChain with an example

In [ ]:
!pip install langchain openai tiktoken chromadb

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

os.environ["OPENAI_API_KEY"] = "your-key"

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Give me a tweet idea on {topic}?",
)

In [3]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("AI"))



AI is changing the way we live! Get ready for the future with AI-powered technology. #AI #FutureTech


Now let's extend the same concept for multiple variables as input

In [4]:
prompt = PromptTemplate(
    input_variables=["topic1", "topic2"],
    template="Give me a tweet idea on {topic1} and {topic2}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'topic1': "AI",
    'topic2': "NLP"
    }))



"AI and NLP are transforming the way we interact with machines. Learn more about the exciting possibilities and implications of this technology!" #AI #NLP


### Sequential Chain

A sequential chain works by combining two or more chains. For example you can take the output from one chain and pass it across to next chains sequentially

Let's try to understand it with the help of an example

First we create a LLMChain to write outline for a blog topic

In [7]:
llm = OpenAI(temperature=.7)
template = """Write a blog outline given a topic.

Topic: {topic}"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
outline_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="outline")

Then we create another LLMChain which will take the output from above chain as input and creates a blog article based on the generated outline

In [11]:
llm = OpenAI(temperature=.7)
template = """Write a blog article based on the below outline.

Outline:
{outline}"""
prompt_template = PromptTemplate(input_variables=["outline"], template=template)
article_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="article")

Finally let's connect both the chains using a SequentialChain which has topic as input and outline, article as output

In [12]:
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[outline_chain, article_chain],
    input_variables=["topic"],
    output_variables=["outline", "article"],
    verbose=True)

In [17]:
overall_chain({"topic":"Deep Learning"})



> Entering new  chain...

> Finished chain.


{'topic': 'Deep Learning',
 'outline': '\n\nI. Introduction \nA. Definition of deep learning \nB. Overview of deep learning \n\nII. History of Deep Learning \nA. Milestones in the development of deep learning \nB. Pioneers in deep learning \n\nIII. Applications of Deep Learning \nA. Uses in computer vision \nB. Uses in natural language processing \nC. Uses in robotics \n\nIV. Advantages of Deep Learning \nA. Increased accuracy \nB. Faster processing times \nC. Lower costs \n\nV. Challenges of Deep Learning \nA. Complexity of algorithms \nB. Availability of data \nC. Lack of interpretability \n\nVI. Conclusion \nA. Summary of deep learning \nB. Impact of deep learning on the future',
 'article': ' \n\nDeep learning has revolutionized the way machines learn and interact with the world. It has a wide range of applications from computer vision to robotics. But what exactly is deep learning and how has it evolved over time? In this article, we’ll explore the definition, history, application

### Retrieval QA chain

Retrieval QA chain is considered one of the most important helping with doing QA over your document data

Here the documents can be pdf, webpages, youtube videos, notion documents etc.

Let's try to understand it with the help of an example

First let's download the data needed to perform QA with

In [19]:
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt

--2023-07-03 12:38:21--  https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0s      

2023-07-03 12:38:21 (97.8 MB/s) - ‘state_of_the_union.txt’ saved [39027/39027]



Let's load the data from text document into Document format in Langchain

In [20]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader('state_of_the_union.txt', encoding='utf8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

Now let's create embeddings needed for this document data

In [ ]:
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

Now let's crete a RetrievalQA chain which can operate QA above the document data along with running a sample query.

As you can see it is able to answer the query based on the document data it is trained on

In [21]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

### LoadSummarize chain

This is the fundamental chain to perform the task of summarization over documents.

This algorithm works by first splitting the entire input into small chunks using a text splitter. Then we create a summary of each of these chunks. Once we get a summary of each, the algorithm creates a summary over these summaries and provides an output

Let's understand it with an example

Let's prepare the text

In [24]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

docs = [Document(page_content=t) for t in texts]

Now let's create a load_summarize chain with chain_type map_reduce . There are other chain types like stuff, refine etc.

In [25]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

" In this speech, the speaker addresses the American people and the world, discussing the recent aggression of Russia's Vladimir Putin in Ukraine and the United States' response. The speaker outlines the economic sanctions and other measures taken to hold Putin accountable, and announces the U.S. Department of Justice's task force to go after the crimes of Russian oligarchs. The speaker also discusses the American Rescue Plan, the Bipartisan Infrastructure Law, and the Bipartisan Innovation Act, and proposes a plan to fight inflation. The speaker also pays tribute to Officer Wilbert Mora and Officer Jason Rivera, who were killed in the line of duty, and nominates Circuit Court of Appeals Judge Ketanji Brown Jackson to the Supreme Court. The speech concludes with a call to fund ARPA-H, the Advanced Research Projects Agency for Health, and a message of optimism for the future of America."

### Router Chain

Router Chain is useful when you have multiple chains for different tasks and you wish to invoke them based on the input provided

For example if you wish to do summuarization / Q&A based on the input provided you can do that with the help of

In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.run("What is black body radiation?"))





> Entering new  chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the electromagnetic radiation emitted by a black body in thermodynamic equilibrium. It is a type of thermal radiation, and is the result of the thermal energy of the body being converted into electromagnetic radiation. The spectrum of the radiation is determined by the temperature of the body, and is a function of the body's emissivity.
